# Import

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load Dataset

In [2]:
DATASET_DIR = "animal_data"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 25
RANDOM_STATE = 42

In [3]:
filepaths = []
labels = []

for class_name in os.listdir(DATASET_DIR):
    class_path = os.path.join(DATASET_DIR, class_name)
    if os.path.isdir(class_path):
        for img in os.listdir(class_path):
            filepaths.append(os.path.join(class_path, img))
            labels.append(class_name)

df = pd.DataFrame({
    "filename": filepaths,
    "class": labels
})

df.head()

,filename,class
0,animal_data\Bear\bear_1.jpg,Bear
1,animal_data\Bear\Bear_10.jpg,Bear
2,animal_data\Bear\Bear_10_1.jpg,Bear
3,animal_data\Bear\Bear_10_2.jpg,Bear
4,animal_data\Bear\Bear_10_3.jpg,Bear


## Train-Validation-Test Split

In [4]:
df_train, df_temp = train_test_split(
    df,
    test_size=0.30,
    stratify=df["class"],
)

df_val, df_test = train_test_split(
    df_temp,
    test_size=0.50,
    stratify=df_temp["class"],
)

In [5]:
datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_gen = datagen.flow_from_dataframe(
    df_train,
    x_col="filename",
    y_col="class",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

val_gen = datagen.flow_from_dataframe(
    df_val,
    x_col="filename",
    y_col="class",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

test_gen = datagen.flow_from_dataframe(
    df_test,
    x_col="filename",
    y_col="class",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

NUM_CLASSES = len(train_gen.class_indices)

Found 1360 validated image filenames belonging to 15 classes.
Found 292 validated image filenames belonging to 15 classes.
Found 292 validated image filenames belonging to 15 classes.


In [7]:
def evaluate_model(model, generator):
    """
    Model Evaluation Function to return the accuracy, precision, recall and F1 Score
    """
    y_true = generator.classes
    y_pred = np.argmax(model.predict(generator), axis=1)

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average="weighted")
    rec = recall_score(y_true, y_pred, average="weighted")
    f1 = f1_score(y_true, y_pred, average="weighted")

    return acc, prec, rec, f1

# Model

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation="relu", input_shape=(224,224,3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation="relu"),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation="relu"),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(256, (3,3), activation="relu"),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(512, activation="relu"),
    layers.Dense(NUM_CLASSES, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
model.fit(train_gen, epochs=EPOCHS, validation_data=val_gen)

# Evaluation

In [ ]:
acc, prec, rec, f1 = evaluate_model(model, test_gen)
print(f"Model Trial - Accuracy: {acc}, Precision: {prec}, Recall: {rec}, F1 Score: {f1}")

NameError: name 'model5' is not defined